In [18]:
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import re

In [16]:
from requests_html import HTMLSession
import nest_asyncio

nest_asyncio.apply()
session = HTMLSession()

In [24]:
listUrl = []
for i in range(0, 4951, 50):
    url = 'https://myanimelist.net/topmanga.php?limit={}'.format(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    listItem = soup.find_all("td", {"class": "title al va-t clearfix word-break"})
    for item in listItem:
        listUrl.append(item.find('a').get('href'))

len(listUrl)

5000

In [50]:
listHtml1 = []
for url in listUrl[:1000]:
    res = session.get(url)
    listHtml1.append(res.text)

In [52]:
listHtml2 = []
for url in listUrl[1000:2000]:
    res = session.get(url)
    listHtml2.append(res.text)

In [53]:
listHtml3 = []
for url in listUrl[2000:3000]:
    res = session.get(url)
    listHtml3.append(res.text)

In [54]:
listHtml4 = []
for url in listUrl[3000:4000]:
    res = session.get(url)
    listHtml4.append(res.text)

In [55]:
listHtml5 = []
for url in listUrl[4000:]:
    res = session.get(url)
    listHtml4.append(res.text)

In [56]:
listHtmlComic = []
listHtmlComic = listHtml1 + listHtml2 + listHtml3 + listHtml4 + listHtml5
len(listHtmlComic)

5000

In [57]:
listHtmlComic[0]

'\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\n    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n\n<html lang="en" xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml" class="appearance-none">\n<head>\n    \n<link rel="preconnect" href="//fonts.gstatic.com/" crossorigin="anonymous" />\n<link rel="preconnect" href="//fonts.googleapis.com/" crossorigin="anonymous" />\n<link rel="preconnect" href="//tags-cdn.deployads.com/" crossorigin="anonymous" />\n<link rel="preconnect" href="//www.googletagservices.com/" crossorigin="anonymous" />\n<link rel="preconnect" href="//www.googletagmanager.com/" crossorigin="anonymous"/>\n<link rel="preconnect" href="//apis.google.com/" crossorigin="anonymous"/>\n<link rel="preconnect" href="//pixel-sync.sitescout.com/" crossorigin="anonymous"/>\n<link rel="preconnect" href="//pixel.tapad.com/" crossorigin="anonymous"/>\n<link rel="preconnect" href="//c.deployads.com/" crossorigin="anonymous"/>\n<link r

In [58]:
ListTitle = []; ListScore = []; ListVote = []; ListRanked = []
ListPopularity = []; ListMember = []; ListFavorite = []; ListVolume = []
ListChapter = []; ListStatus = []; ListPublished = []; ListGenres = []
ListThemes = []; ListAuthor = []; ListTotalReview = []; ListTypeReview = []
count = 0
for htmlComic in listHtmlComic:
    soup = BeautifulSoup(htmlComic, "html.parser")
    print(count)
    count += 1
    if count == 27:
        print(soup)
    title = soup.find('span', {'itemprop' : 'name'})
    if (title is None):
        continue
    title = title.text

    ratingValue = soup.find('span', {'itemprop' : 'ratingValue'}).text

    ratingCount = soup.find('span', {'itemprop' : 'ratingCount'}).text

    ranked = soup.find('span', {'class' : 'numbers ranked'}).text
    ranked = re.findall(r'\d+', ranked)[0]

    popularity = soup.find('span', {'class' : 'numbers popularity'}).text
    popularity = re.findall(r'\d+', popularity)[0]

    listSpaceit = soup.find_all("div", {'class': 'spaceit_pad'})
    volumes = ''; chapters = ''; status = ''; published = ''
    genres = []; themes = []; authors = ''; favorites = ''; members = ''
    for space in listSpaceit:
        if 'Volumes' in space.text:
            volumes = space.text.split(':')[1].strip()
        if 'Chapters' in space.text:
            chapters = space.text.split(':')[1].strip()
        if 'Status' in space.text:
            status = space.text.split(':')[1].strip()
        if 'Published' in space.text:
            published = space.text.split(':')[1].strip()
        if 'Genres' in space.text:
            listGenres = space.find_all('span', {'itemprop': 'genre'})
            [genres.append(gen.text) for gen in listGenres]
        if 'Themes' in space.text:
            listGenre = space.find_all('span', {'itemprop': 'genre'})
            [themes.append(theme.text) for theme in listGenre]
        if 'Authors' in space.text:
            authors = space.text.split(':')[1].strip()
        if 'Favorites' in space.text:
            favorites = space.text.split(':')[1].strip()
        if 'Members' in space.text:
            members = space.text.split(':')[1].strip()


    infoReviews = soup.find('div', {'class': 'manga-info-review__header mal-navbar'})

    totalReviews = infoReviews.find('div',{'class': 'right'}).text
    totalReviews = re.findall(r'\d+', totalReviews)[0]

    typeReview = [] # 3 type: recommended, mixed feelings, not recommended
    type = infoReviews.find('div',{'class': 'center'})

    recommended = re.findall(r'\d+', type.find('div',{'class': 'recommended'}).text)[0]
    typeReview.append(recommended)
    mixed = re.findall(r'\d+', type.find('div',{'class': 'mixed-feelings'}).text)[0]
    typeReview.append(mixed)
    notRecommended = re.findall(r'\d+', type.find('div',{'class': 'not-recommended'}).text)[0]
    typeReview.append(notRecommended)

    ListTitle.append(title); ListScore.append(ratingValue); ListVote.append(ratingCount)
    ListRanked.append(ranked); ListPopularity.append(popularity); ListMember.append(members)
    ListFavorite.append(favorites); ListVolume.append(volumes); ListChapter.append(chapters)
    ListStatus.append(status); ListPublished.append(published); ListGenres.append(genres)
    ListThemes.append(themes); ListAuthor.append(authors); ListTotalReview.append(totalReviews)
    ListTypeReview.append(typeReview)

df = pd.DataFrame({"Title": ListTitle, "Score": ListScore,
                   "Vote": ListVote, "Ranked": ListRanked,
                   "Popularity": ListPopularity, "Members": ListMember,
                   "Favorite": ListFavorite, "Volumes": ListVolume,
                   "Chapters": ListChapter, "Status": ListStatus,
                   "Published": ListPublished, "Genres": ListGenres,
                   "Themes": ListThemes, "Author": ListAuthor,
                   "Total Review": ListTotalReview, "Type Review": ListTypeReview})

df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html class="appearance-none" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<link crossorigin="anonymous" href="//fonts.gstatic.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//fonts.googleapis.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//tags-cdn.deployads.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//www.googletagservices.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//www.googletagmanager.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//apis.google.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//pixel-sync.sitescout.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//pixel.tapad.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="/

,Title,Score,Vote,Ranked,Popularity,Members,Favorite,Volumes,Chapters,Status,Published,Genres,Themes,Author,Total Review,Type Review
0,Berserk,9.47,330305,1,1,"663,547","122,563",Unknown,Unknown,Publishing,"Aug 25, 1989 to ?","[Action, Adventure, Award Winning, Drama, Fant...","[Gore, Military, Mythology, Psychological]","Miura, Kentarou (Story & Art), Studio Gaga (Art)",259,"[234, 15, 10]"
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,9.30,155971,2,26,"255,530","42,788",24,96,Finished,"Jan 19, 2004 to Apr 19, 2011","[Action, Adventure, Mystery, Supernatural]",[],"Araki, Hirohiko (Story & Art)",128,"[120, 7, 1]"
2,Vagabond,9.24,135838,3,15,"363,657","40,000",37,327,On Hiatus,"Sep 3, 1998 to May 21, 2015","[Action, Adventure, Award Winning]","[Historical, Samurai]","Inoue, Takehiko (Story & Art), Yoshikawa, Eiji...",95,"[87, 7, 1]"
3,One Piece,9.22,365890,4,3,"597,907","114,303",Unknown,Unknown,Publishing,"Jul 22, 1997 to ?","[Action, Adventure, Fantasy]",[],"Oda, Eiichiro (Story & Art)",206,"[173, 17, 16]"
4,Monster,9.15,93634,5,29,"235,688","20,467",18,162,Finished,"Dec 5, 1994 to Dec 20, 2001","[Award Winning, Drama, Mystery]","[Adult Cast, Psychological]","Urasawa, Naoki (Story & Art)",76,"[64, 7, 5]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Futari no Himitsu,7.27,959,4995,7012,"2,589",9,3,11,Finished,"Mar 3, 2011 to Dec 28, 2011",[],[],"Moro, Orie (Story & Art)",1,"[1, 0, 0]"
4996,Hoshi wo Tsugu Mono,7.27,204,4996,14158,"1,021",0,4,36,Finished,"Feb 25, 2011 to Aug 10, 2012","[Award Winning, Mystery, Sci-Fi]",[],"Hoshino, Yukinobu (Art), Hogan, James Patrick ...",1,"[1, 0, 0]"
4997,Aoharu x Kikanjuu,7.27,1210,4997,6666,"2,749",22,Unknown,1,Finished,"Jul 28, 2011","[Action, Comedy, Slice of Life]",[],NAOE (Story & Art),1,"[1, 0, 0]"
4998,Minamoto-kun Monogatari,7.27,22417,4998,277,"56,837","1,296",16,360,Finished,"Sep 15, 2011 to Sep 5, 2019","[Romance, Ecchi]","[Harem, School]","Inaba, Minori (Story & Art)",19,"[14, 5, 0]"


In [59]:
df.to_csv('./data/comic.csv', index=False)